In [41]:
import os

# Save the original working directory
original_dir = os.getcwd()

# Move working directory one level up
os.chdir(os.path.abspath(os.path.join(original_dir, "..")))
print("New working directory:", os.getcwd())
# Install dependencies
!pip install -r requirements.txt --quiet

New working directory: g:\Other computers\Flip15\UWA\3rd Semester\Data Warehousing\PROJECT WH\DataWH_project


In [42]:
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Load environment variables from .env file, remember to create this file with your credentials
# Read readme file for more information ! 
load_dotenv("postgressql_secretkeys.env")  
# Extract variables
username = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host     = os.getenv("PG_HOST")
port     = os.getenv("PG_PORT")
dbname   = os.getenv("PG_DB")

#incase the above code does not work, you can uncommet the below and pass your credentials directly
#username = 'postgres'
#password = '123'
#host = 'localhost'
#port = '5432'
#dbname = 'DWH_UWA'
# Create the SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}")

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    print(result.fetchone())
    print("======================================= \n || -- Connected to PostgreSQL! -- || \n=======================================")


('PostgreSQL 17.4 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit',)
 || -- Connected to PostgreSQL! -- || 


In [43]:
import pandas as pd
# Query a full table with all dimensions and fact from data warehouse query for data mining

query = """
SELECT * 
FROM fact_fatality
JOIN dim_crash_type ON fact_fatality.crash_type_sk = dim_crash_type.crash_type_sk
JOIN dim_gender ON fact_fatality.gender_sk = dim_gender.gender_sk
JOIN dim_speed_limit ON fact_fatality.speed_limit_sk = dim_speed_limit.speed_limit_sk
JOIN dim_age ON fact_fatality.age_sk = dim_age.age_sk
JOIN dim_road_user ON fact_fatality.road_user_sk = dim_road_user.road_user_sk
JOIN dim_road_type ON fact_fatality.road_type_sk = dim_road_type.road_type_sk
JOIN dim_time ON fact_fatality.time_sk = dim_time.time_sk
JOIN dim_location ON fact_fatality.state_id = dim_location.state_id
"""

data = pd.read_sql(text(query), con=engine)
data.head(10)

,fatality_sk,crash_type_sk,gender_sk,speed_limit_sk,age_sk,road_user_sk,road_type_sk,time_sk,state_id,crash_type_sk,...,road_type_sk,road_type,time_sk,time,dayweek,month,year,state_id,population,state_name
0,1,1,1,1,1,1,1,1,NSW,1,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
1,2,3,1,1,1,1,1,1,NSW,3,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
2,3,5,1,1,1,1,1,1,NSW,5,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
3,4,8,1,1,1,1,1,1,NSW,8,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
4,5,9,1,1,1,1,1,1,NSW,9,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
5,6,11,1,1,1,1,1,1,NSW,11,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
6,7,13,1,1,1,1,1,1,NSW,13,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
7,8,16,1,1,1,1,1,1,NSW,16,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
8,9,18,1,1,1,1,1,1,NSW,18,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales
9,10,19,1,1,1,1,1,1,NSW,19,...,1,Arterial Road,1,04,Friday,12,2024,NSW,8342285,New South Wales


In [44]:
# Drop all the columns that are not needed for the analysis
data.drop(columns=[
    'crash_type_sk',
    'gender_sk',
    'speed_limit_sk',
    'age_sk',
    'road_user_sk',
    'road_type_sk',
    'time_sk',
    'state_id',
    'fatality_sk'
], inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593982 entries, 0 to 593981
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   crash_type           593982 non-null  object
 1   fatalities_by_crash  593982 non-null  int64 
 2   gender               593982 non-null  object
 3   speed_limit          593982 non-null  object
 4   age                  593982 non-null  object
 5   age_group            593982 non-null  object
 6   road_user            593982 non-null  object
 7   road_type            593982 non-null  object
 8   time                 593549 non-null  object
 9   dayweek              593982 non-null  object
 10  month                593982 non-null  object
 11  year                 593982 non-null  object
 12  population           593982 non-null  int64 
 13  state_name           593982 non-null  object
dtypes: int64(2), object(12)
memory usage: 63.4+ MB


In [45]:
#Check missing values in the dataframe
data.isna()

#Count total missing values at each column in the dataframe
data.isna().sum()

crash_type               0
fatalities_by_crash      0
gender                   0
speed_limit              0
age                      0
age_group                0
road_user                0
road_type                0
time                   433
dayweek                  0
month                    0
year                     0
population               0
state_name               0
dtype: int64

## Data Trasformation 

In [46]:
# Tranform the data to be used in the mining process using apriori algorithm
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

#Data transformation to str and then to list
#Convert all boolean values to column names and values Yes or No
data = data.astype(str)
data = data.apply(lambda row: [f"{col}={val}" for col, val in row.items()], axis=1)

#TransactionEncoder() was designed to covert lists to array
list_ = data.values.tolist()

#Covert the list to one-hot encoded boolean numpy array. 
#Apriori function allows boolean data type only, such as 1 and 0, or FALSE and TRUE.
te = TransactionEncoder()
array_te = te.fit(list_).transform(list_)

#Check the array
array_te

#Check the colunms
te.columns_

#Apriori function can handle dataframe only, covert the array to a dataframe
arm_df = pd.DataFrame(array_te, columns = te.columns_)
arm_df

,age=0,age=1,age=10,age=100,age=101,age=11,age=12,age=13,age=14,age=15,...,year=2015,year=2016,year=2017,year=2018,year=2019,year=2020,year=2021,year=2022,year=2023,year=2024
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593977,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
593978,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
593979,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
593980,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## FP-Growth Algorithm 

In [47]:
# Run FP-Growth for frequent itemsets
# Hyperparameters support = 0.15 and confidence = 0.5

# Apply support value at 0.15
min_support_value = 0.15  # Adjust this value based on your analysis
frequent_itemsets_fp = fpgrowth(arm_df, min_support=min_support_value, use_colnames=True)

# Generate association rules with confidence threshold as > 0.5
min_confidence_value = 0.5  # Adjust this value based on your analysis
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold = min_confidence_value)
rules_fp

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(road_type=Unknown),(gender=Male),0.811173,0.716505,0.577371,0.711773,0.993396,-0.003838,0.983584
1,(gender=Male),(road_type=Unknown),0.716505,0.811173,0.577371,0.805816,0.993396,-0.003838,0.972414
2,(crash_type=Single),(gender=Male),0.532541,0.716505,0.397032,0.745542,1.040527,0.015464,1.114116
3,(gender=Male),(crash_type=Single),0.716505,0.532541,0.397032,0.554124,1.040527,0.015464,1.048404
4,(crash_type=Single),(road_type=Unknown),0.532541,0.811173,0.432235,0.811646,1.000584,0.000252,1.002515
...,...,...,...,...,...,...,...,...,...
112,(speed_limit=60),(gender=Male),0.253235,0.716505,0.180729,0.713683,0.996061,-0.000715,0.990143
113,(speed_limit=60),(road_type=Unknown),0.253235,0.811173,0.220438,0.870487,1.073121,0.015020,1.457976
114,"(road_type=Unknown, speed_limit=60)",(gender=Male),0.220438,0.716505,0.156567,0.710255,0.991278,-0.001378,0.978431
115,"(speed_limit=60, gender=Male)",(road_type=Unknown),0.180729,0.811173,0.156567,0.866306,1.067968,0.009964,1.412389


In [48]:

# Filtering the result to show only "Road User" on the rigth hand side (consequents)
result_fp = rules_fp[['antecedents','consequents','support','confidence','lift']]
filtered_rules_road_users = result_fp[
    result_fp['consequents'].apply(lambda x: len(x) == 1 and next(iter(x)).startswith('road_user'))
]

#Sort the output with lift then confidence 
ranked_rules = filtered_rules_road_users.sort_values(by=['lift', 'confidence'], ascending=[False, False])

#display top 5 rule of associate found 
ranked_rules.head(5)


,antecedents,consequents,support,confidence,lift
23,(speed_limit=100),(road_user=Driver),0.195738,0.569748,1.258188
29,"(road_type=Unknown, speed_limit=100)",(road_user=Driver),0.158641,0.560880,1.238604


In [49]:
#incase you want to generate csv file to view data 
flag = False
if flag:
    ranked_rules.to_csv('ranked_road_user_rules_015.csv', index=False)

In [50]:
# Restore original working directory
os.chdir(original_dir)
print("Restored working directory:", os.getcwd())

Restored working directory: g:\Other computers\Flip15\UWA\3rd Semester\Data Warehousing\PROJECT WH\DataWH_project\Data Mining
